In [61]:
import time
import os

import scanpy as sc
import numpy as np
import scipy.sparse as sp

import torch
from torch import optim
from torch.utils.data import DataLoader

import models.loadImg as loadImg
import models.modelsCNN as modelsCNN
import models.optimizer as optimizer

import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import gc
from skimage import io
import scipy.stats

import statsmodels.stats.multitest as multitest
import networkx as nx

In [11]:
#load model
os.environ["CUDA_LAUNCH_BLOCKING"] = "2" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2" 
use_cuda=True
datadir='/media/xinyi/dcis2idc/data'
name='exp0'
plotsavepath='/media/xinyi/dcis2idc/plots/cnnvae'+name
plottype='umap'
sampledir=plotsavepath
savedir=os.path.join(sampledir,'embedding_'+plottype)
clustersavedir=os.path.join(sampledir,'cluster')

with open(os.path.join(datadir,'processed','train_cnnvae_names'), 'rb') as input:
    allImgNames=pickle.load(input)


br1003aSpecs=pd.read_excel('/media/xinyi/dcis2idc/data/BR1003a specs.xlsx',header=10)
br301Specs=pd.read_excel('/media/xinyi/dcis2idc/data/BR301 specs.xlsx',header=10)
br8018aSpecs=pd.read_excel('/media/xinyi/dcis2idc/data/BR8018a specs.xlsx',header=10)
br1003aSpecs.index=br1003aSpecs.loc[:,'Position']
br301Specs.index=br301Specs.loc[:,'Position']
br8018aSpecs.index=br8018aSpecs.loc[:,'Position']
progList=np.copy(allImgNames)
for s in np.unique(allImgNames):
    ssplit=s.split('_')
    if 'br1003a'==ssplit[0]:
        prog_s=br1003aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br301'==ssplit[0]:
        prog_s=br301Specs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br8018a'==ssplit[0]:
        prog_s=br8018aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    progList[allImgNames==s]=prog_s
    
progNames,progCounts=np.unique(progList,return_counts=True)
progSampleRate={}
for p in range(progNames.size):
    progSampleRate[progNames[p]]=np.min(progCounts)/progCounts[p]
    
ep=311
np.random.seed(6)
# plotPCT=4500
plottingIdx_i=np.array([])
n_pcs=50
uniqueImgNames,imgNameIdx=np.unique(allImgNames,return_index=True)
for i in range(1):
    for sidx in range(uniqueImgNames.size):
        s=uniqueImgNames[sidx]
        p=progList[imgNameIdx[sidx]]
        print(s+' '+p)
        nsamples=int(np.sum(allImgNames==s)*progSampleRate[p])
        plottingIdx_i=np.concatenate((plottingIdx_i,
                                    np.random.choice(np.arange(allImgNames.shape[0])[allImgNames==s],nsamples,replace=False)))
    
ncluster=8
savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(ep)+'_plottingIdx_progBalanced_'+str(i)
with open(os.path.join(clustersavedir,savenamecluster), 'rb') as output:
    clusterRes=pickle.load(output)

plotsavenameAdd='_plottingIdx_progBalanced_'+str(i)

subclusternumbers=[4,6,8,6,6,6,6,4]
clusterRes_sub=np.zeros(clusterRes.size)-1
for i in np.unique(clusterRes):
    subcluster=subclusternumbers[i]
    subclustersavedir=os.path.join(clustersavedir,savenamecluster+'_subcluster'+str(i))

    savenameclustersub='minibatchkmean_ncluster'+str(subcluster)+'n_pcs'+str(n_pcs)+'epoch'+str(ep)+plotsavenameAdd
    with open(os.path.join(subclustersavedir,savenameclustersub), 'rb') as output:
        subclusterRes=pickle.load(output)
        
    clusterRes_sub[clusterRes==i]=subclusterRes

print(np.sum(clusterRes_sub==-1))
    
with open(os.path.join(datadir,'processed','train_cnnvae_cellLabels'), 'rb') as output:
    cellIDlist=pickle.load(output)
    
#load NMCO
uniquenames,nameIdx=np.unique(allImgNames,return_index=True)

allstats=None
alllabels=None
alllabels_sub=None
allvarnames=None
for sidx in range(uniquenames.size):
    s=np.unique(allImgNames)[sidx]

    path_s=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'nmco_features',s.split('_')[-1] +'.csv')
    if not os.path.exists(path_s):
        print('DNE '+path_s)
    print(s)
    plottingIdx_i_s=plottingIdx_i.astype(int)[allImgNames[plottingIdx_i.astype(int)]==s]-nameIdx[sidx]
    assert np.min(plottingIdx_i_s)>=0

    stats_s=pd.read_csv(path_s)
    stats_s.index=stats_s.loc[:,'label']
    stats_s=stats_s.loc[cellIDlist[s][plottingIdx_i_s]].to_numpy()[:,2:-2]
    print(stats_s.shape)


#         ssplit=s.split('_')
    slabels=clusterRes[allImgNames[plottingIdx_i.astype(int)]==s]
    slabels_sub=clusterRes_sub[allImgNames[plottingIdx_i.astype(int)]==s]

    if allstats is None:
        allstats=stats_s
        alllabels=np.copy(slabels)
        alllabels_sub=np.copy(slabels_sub)
    else:
        allstats=np.concatenate((allstats,stats_s),axis=0)
        alllabels=np.concatenate((alllabels,np.copy(slabels)))
        alllabels_sub=np.concatenate((alllabels_sub,np.copy(slabels_sub)))



br1003a_1_cytokeratin_555_aSMA_647_hoechst_A1 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A2 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A4 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A5 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A6 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A7 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A8 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A9 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C1 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C10 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C2 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C3 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C4 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C5 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C6 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C7 Hyperplasia
br1003a_1_cytokeratin_555_

br301_6_collagen1_647_hoechst_D2 Ductal carcinoma in situ with early infiltratio
br301_6_collagen1_647_hoechst_D3 Invasive ductal carcinoma and breast tissue
br301_6_collagen1_647_hoechst_D4 Invasive ductal carcinoma
br301_6_collagen1_647_hoechst_D5 Ductal carcinoma in situ with early infiltratio
br301_6_collagen1_647_hoechst_D6 Ductal carcinoma in situ with early infiltratio
br301_6_collagen1_647_hoechst_E1 Ductal carcinoma in situ with early infiltratio
br301_6_collagen1_647_hoechst_E2 Ductal carcinoma in situ with early infiltratio
br301_6_collagen1_647_hoechst_E3 Invasive ductal carcinoma and breast tissue
br301_6_collagen1_647_hoechst_E4 Invasive ductal carcinoma
br301_6_collagen1_647_hoechst_E5 Ductal carcinoma in situ with early infiltratio
br301_6_collagen1_647_hoechst_E6 Ductal carcinoma in situ with early infiltratio
br8018a_1_cytokeratin_555_aSMA_647_hoechst_A1 Invasive ductal carcinoma
br8018a_1_cytokeratin_555_aSMA_647_hoechst_A10 Invasive ductal carcinoma
br8018a_1_cytoke

br1003a_3_collagen1_647_hoechst_A7
(716, 201)
br1003a_3_collagen1_647_hoechst_A8
(481, 201)
br1003a_3_collagen1_647_hoechst_A9
(274, 201)
br1003a_3_collagen1_647_hoechst_C1
(2720, 201)
br1003a_3_collagen1_647_hoechst_C10
(779, 201)
br1003a_3_collagen1_647_hoechst_C2
(1286, 201)
br1003a_3_collagen1_647_hoechst_C3
(1172, 201)
br1003a_3_collagen1_647_hoechst_C4
(2233, 201)
br1003a_3_collagen1_647_hoechst_C5
(1350, 201)
br1003a_3_collagen1_647_hoechst_C6
(1136, 201)
br1003a_3_collagen1_647_hoechst_C7
(890, 201)
br1003a_3_collagen1_647_hoechst_C8
(556, 201)
br1003a_3_collagen1_647_hoechst_C9
(824, 201)
br1003a_3_collagen1_647_hoechst_I1
(1003, 201)
br1003a_3_collagen1_647_hoechst_I10
(1564, 201)
br1003a_3_collagen1_647_hoechst_I2
(1201, 201)
br1003a_3_collagen1_647_hoechst_I3
(1379, 201)
br1003a_3_collagen1_647_hoechst_I7
(1300, 201)
br1003a_3_collagen1_647_hoechst_I8
(1609, 201)
br1003a_3_collagen1_647_hoechst_I9
(853, 201)
br1003a_4_cytokeratin_555_gh2ax_647_hoechst_A1
(253, 201)
br1003a_

(7843, 201)
br8018a_3_collagen1_647_hoechst_B7
(351, 201)
br8018a_3_collagen1_647_hoechst_B8
(230, 201)
br8018a_3_collagen1_647_hoechst_B9
(339, 201)
br8018a_3_collagen1_647_hoechst_F10
(467, 201)
br8018a_3_collagen1_647_hoechst_F3
(202, 201)
br8018a_3_collagen1_647_hoechst_F4
(362, 201)
br8018a_3_collagen1_647_hoechst_F6
(297, 201)
br8018a_3_collagen1_647_hoechst_F7
(341, 201)
br8018a_3_collagen1_647_hoechst_F8
(476, 201)
br8018a_3_collagen1_647_hoechst_F9
(419, 201)
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_A2
(259, 201)
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_A3
(453, 201)
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_A5
(150, 201)
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_A7
(488, 201)
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_A8
(211, 201)
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_B4
(484, 201)
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_B5
(276, 201)
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_B6
(8272, 201)
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_B7
(356, 201)
br80

In [12]:
stats_s=pd.read_csv(path_s)
nmcoNames=np.array(stats_s.columns[2:-2])

In [19]:
seed=3
epochs=6000
saveFreq=200
lr=0.001 #initial learning rate
weight_decay=0 #Weight for L2 loss on embedding matrix.

# batchsize=4
batchsize=8000
model_str='fc3'

kernel_size=4
stride=2
padding=1

fc_dim1=128
fc_dim2=128
fc_dim3=128


dropout=0.01
kl_weight=0.0000001

name='exp0_clusterClf_nmco_subclusters_'+savenamecluster+'fcl3'
logsavepath='/media/xinyi/dcis2idc/log/cnnvae'+name
modelsavepath='/media/xinyi/dcis2idc/models/cnnvae'+name
plotsavepath='/media/xinyi/dcis2idc/plots/cnnvae'+name



In [59]:
allstats=allstats.astype(float)
allstats=scipy.stats.zscore(allstats,axis=0,nan_policy='omit')
allstats=np.nan_to_num(allstats,nan=0)

for i in np.unique(clusterRes):
    print(i)
    logsavepath_sub=os.path.join(logsavepath,str(i))
    plotsavepath_sub=os.path.join(plotsavepath,str(i))
    modelsavepath_sub=os.path.join(modelsavepath,str(i))
    
    stats_sub=allstats[alllabels==i]
    labels_sub=alllabels_sub[alllabels==i].astype(int)
    
    # Create model
    seed=3
    torch.manual_seed(seed)
    nclasses=np.unique(labels_sub).size
    if use_cuda:
        torch.cuda.manual_seed(seed)

    if model_str=='fc3':
        model = modelsCNN.FC_l3(allstats.shape[1],fc_dim1,fc_dim2,fc_dim3,nclasses,0.5,regrs=False)
    if model_str=='fc5':
        model = modelsCNN.FC_l5(allstats.shape[1],fc_dim1,fc_dim2,fc_dim3,fc_dim4,fc_dim5,nclasses,0.5,regrs=False)
    if model_str=='fc1':
        model = modelsCNN.FC_l1(allstats.shape[1],fc_dim1,nclasses,regrs=False)
    if model_str=='fc0':
        model = modelsCNN.FC_l0(allstats.shape[1],nclasses,regrs=False)

    if use_cuda:
        model.cuda()
    model.load_state_dict(torch.load(os.path.join(modelsavepath_sub,str(5800)+'.pt')))

    model.eval()

    trainIdx=np.arange(stats_sub.shape[0])
    allgrad_byLabel=np.zeros_like(stats_sub)
    for i in range(int(np.ceil(trainIdx.shape[0]/batchsize))):
        trainIdx_i=trainIdx[i*batchsize:min((i+1)*batchsize,trainIdx.shape[0])]

    #         trainInput=trainInputnp[trainIdx]
    #         labels=trainLabelsnp[trainIdx]
        if use_cuda:
            trainInput=torch.tensor(stats_sub[trainIdx_i]).cuda().float()
        trainInput.requires_grad = True
        model.zero_grad()
        pred = model(trainInput)

        for gidx in range(trainIdx_i.size):
            grad=torch.autograd.grad(pred[gidx,labels_sub[trainIdx_i[gidx]]],trainInput,retain_graph=True)[0][gidx]
            allgrad_byLabel[trainIdx_i[gidx]]=grad.cpu().numpy()
    
    gradsavepath=os.path.join(plotsavepath_sub,'grad')
    if not os.path.exists(gradsavepath):
        os.mkdir(gradsavepath)
    
    gradAvg=np.zeros((np.unique(labels_sub).size,nmcoNames.size))
    for l in np.unique(labels_sub):
        gradAvg[l]=np.mean(allgrad_byLabel[labels_sub==l],axis=0)
    
    fig, ax = plt.subplots(figsize=(100, 10))
    im = ax.imshow(gradAvg,cmap='seismic',vmin=-np.max(np.abs(gradAvg)),vmax=np.max(np.abs(gradAvg)))
    ax.set_xticks(np.arange(nmcoNames.size))
    ax.set_xticklabels(nmcoNames)
    ax.set_yticks(np.arange(np.unique(labels_sub).size))
    ax.set_yticklabels(np.unique(labels_sub))
    plt.setp(ax.get_xticklabels(), rotation=90, ha="right",rotation_mode="anchor")
    #     fig.set_figheight(35)
    #     fig.set_figwidth(35)
    fig.colorbar(im)
    fig.tight_layout()
    plt.savefig(os.path.join(gradsavepath,'all.jpg'))
    plt.close()
    
    nmcoSigIdx=np.array([])
    stats={}
    pvals=np.array([])
    pval_startIdx=[]
    currIdx=0
    for l in np.unique(labels_sub):
        stats[l]=pd.DataFrame({'avg':np.mean(allgrad_byLabel[labels_sub==l],axis=0),
                           'std':np.std(allgrad_byLabel[labels_sub==l],axis=0),
                           'fc':(np.mean(allgrad_byLabel[labels_sub==l],axis=0)/np.mean(allgrad_byLabel[labels_sub!=l],axis=0))},
                          index=nmcoNames)
        pvalues=scipy.stats.ttest_ind(allgrad_byLabel[labels_sub==l],allgrad_byLabel[labels_sub!=l],equal_var=False)[1]
        stats[l]['pval']=pvalues
        pvals=np.concatenate((pvals,pvalues))
        pval_startIdx.append(currIdx)
        currIdx+=pvalues.size

    pval_corr=multitest.fdrcorrection(pvals,0.05,'indep')[1]

    for l in np.unique(labels_sub):
        stats[l]['pval_corr']=pval_corr[pval_startIdx[l]:(pval_startIdx[l]+stats[l]['pval'].size)]
        stats[l].to_csv(os.path.join(gradsavepath,str(l)+'_stats.csv'))
        nmcoSigIdx=np.concatenate((nmcoSigIdx,np.arange(stats[l].shape[0])[np.logical_and(np.logical_or(stats[l]['fc']<0.8,stats[l]['fc']>1.2),stats[l]['pval_corr']<0.05)]))

    nmcoSigIdx=np.unique(nmcoSigIdx).astype(int)
    print(nmcoSigIdx.size)
    fig, ax = plt.subplots(figsize=(100, 10))
    im = ax.imshow(gradAvg[:,nmcoSigIdx],cmap='seismic',vmin=-np.max(np.abs(gradAvg[:,nmcoSigIdx])),vmax=np.max(np.abs(gradAvg[:,nmcoSigIdx])))
    ax.set_xticks(np.arange(nmcoSigIdx.size))
    ax.set_xticklabels(nmcoNames[nmcoSigIdx])
    ax.set_yticks(np.arange(np.unique(labels_sub).size))
    ax.set_yticklabels(np.unique(labels_sub))
    plt.setp(ax.get_xticklabels(), rotation=90, ha="right",rotation_mode="anchor")
    #     fig.set_figheight(35)
    #     fig.set_figwidth(35)
    fig.colorbar(im)
    fig.tight_layout()
    plt.savefig(os.path.join(gradsavepath,'all_pval05_fc1.2.jpg'))
    plt.close()

0
201
1
201
2
201
3
201
4
201
5
201
6
201
7
201


In [56]:
#also test NMCO difference directly

for i in np.unique(clusterRes):
    print(i)
    logsavepath_sub=os.path.join(logsavepath,str(i))
    plotsavepath_sub=os.path.join(plotsavepath,str(i))
    modelsavepath_sub=os.path.join(modelsavepath,str(i))
    
    nmcodesavepath=os.path.join(plotsavepath_sub,'nmcode')
    if not os.path.exists(nmcodesavepath):
        os.mkdir(nmcodesavepath)
    
    stats_sub=allstats[alllabels==i]
    labels_sub=alllabels_sub[alllabels==i].astype(int)
    
    
    nmcoAvg=np.zeros((np.unique(labels_sub).size,nmcoNames.size))
    for l in np.unique(labels_sub):
        nmcoAvg[l]=np.mean(stats_sub[labels_sub==l],axis=0)

    fig, ax = plt.subplots(figsize=(100, 10))
    im = ax.imshow(nmcoAvg,cmap='seismic',vmin=-np.max(np.abs(nmcoAvg)),vmax=np.max(np.abs(nmcoAvg)))
    ax.set_xticks(np.arange(nmcoNames.size))
    ax.set_xticklabels(nmcoNames)
    ax.set_yticks(np.arange(np.unique(labels_sub).size))
    ax.set_yticklabels(np.unique(labels_sub))
    plt.setp(ax.get_xticklabels(), rotation=90, ha="right",rotation_mode="anchor")
    #     fig.set_figheight(35)
    #     fig.set_figwidth(35)
    fig.colorbar(im)
    fig.tight_layout()
    plt.savefig(os.path.join(nmcodesavepath,'all.jpg'))
    plt.close()
    
    nonzeroIdx=np.sum(stats_sub==0,axis=0)==0
    allstats_nonzero=stats_sub[:,nonzeroIdx]

    nmcoSigIdx=np.array([])
    stats={}
    pvals=np.array([])
    pval_startIdx=[]
    currIdx=0
    for l in np.unique(labels_sub):
        stats[l]=pd.DataFrame({'avg':np.mean(allstats_nonzero[labels_sub==l],axis=0),
                           'std':np.std(allstats_nonzero[labels_sub==l],axis=0),
                           'fc':(np.mean(allstats_nonzero[labels_sub==l],axis=0)/np.mean(allstats_nonzero[labels_sub!=l],axis=0))},
                          index=nmcoNames[nonzeroIdx])
        pvalues=scipy.stats.ttest_ind(allstats_nonzero[labels_sub==l],allstats_nonzero[labels_sub!=l],equal_var=False)[1]
        stats[l]['pval']=pvalues
        pvals=np.concatenate((pvals,pvalues))
        pval_startIdx.append(currIdx)
        currIdx+=pvalues.size

    pval_corr=multitest.fdrcorrection(pvals,0.05,'indep')[1]

    for l in np.unique(labels_sub):
        stats[l]['pval_corr']=pval_corr[pval_startIdx[l]:(pval_startIdx[l]+stats[l]['pval'].size)]
        stats[l].to_csv(os.path.join(nmcodesavepath,str(l)+'_stats05.csv'))
        nmcoSigIdx=np.concatenate((nmcoSigIdx,np.arange(stats[l].shape[0])[np.logical_and(np.abs(stats[l]['avg'])>0.5,np.logical_and(stats[l]['pval_corr']<0.01,np.logical_or(stats[l]['fc']<0.8,stats[l]['fc']>1.2)))]))

    nmcoSigIdx=np.unique(nmcoSigIdx).astype(int)
    print(nmcoSigIdx.size)
    fig, ax = plt.subplots(figsize=(100, 10))
    im = ax.imshow(nmcoAvg[:,nonzeroIdx][:,nmcoSigIdx],cmap='seismic',vmin=-np.max(np.abs(nmcoAvg[:,nonzeroIdx][:,nmcoSigIdx])),vmax=np.max(np.abs(nmcoAvg[:,nonzeroIdx][:,nmcoSigIdx])))
    ax.set_xticks(np.arange(nmcoSigIdx.size))
    ax.set_xticklabels(nmcoNames[nonzeroIdx][nmcoSigIdx])
    ax.set_yticks(np.arange(np.unique(labels_sub).size))
    ax.set_yticklabels(np.unique(labels_sub))
    plt.setp(ax.get_xticklabels(), rotation=90, ha="right",rotation_mode="anchor")
    #     fig.set_figheight(35)
    #     fig.set_figwidth(35)
    fig.colorbar(im)
    fig.tight_layout()
    plt.savefig(os.path.join(nmcodesavepath,'all_pval05_fc1.2_avg05.jpg'))
    plt.close()

0
113
1
87
2
88
3
105
4
129
5
97
6
139
7
51


In [66]:
#also test NMCO difference directly

for i in np.unique(clusterRes):
    print(i)
    logsavepath_sub=os.path.join(logsavepath,str(i))
    plotsavepath_sub=os.path.join(plotsavepath,str(i))
    modelsavepath_sub=os.path.join(modelsavepath,str(i))
    
    nmcodesavepath=os.path.join(plotsavepath_sub,'nmcode')
    if not os.path.exists(nmcodesavepath):
        os.mkdir(nmcodesavepath)
    
    stats_sub=allstats[alllabels==i]
    labels_sub=alllabels_sub[alllabels==i].astype(int)
    
    
    nmcoAvg=np.zeros((np.unique(labels_sub).size,nmcoNames.size))
    for l in np.unique(labels_sub):
        nmcoAvg[l]=np.mean(stats_sub[labels_sub==l],axis=0)

    
    nonzeroIdx=np.sum(stats_sub==0,axis=0)==0
    allstats_nonzero=stats_sub[:,nonzeroIdx]

    nmcoSigIdx=np.array([])
    stats={}
    pvals=np.array([])
    pval_startIdx=[]
    currIdx=0
    for l in np.unique(labels_sub):
        stats[l]=pd.DataFrame({'avg':np.mean(allstats_nonzero[labels_sub==l],axis=0),
                           'std':np.std(allstats_nonzero[labels_sub==l],axis=0),
                           'fc':(np.mean(allstats_nonzero[labels_sub==l],axis=0)/np.mean(allstats_nonzero[labels_sub!=l],axis=0))},
                          index=nmcoNames[nonzeroIdx])
        pvalues=scipy.stats.ttest_ind(allstats_nonzero[labels_sub==l],allstats_nonzero[labels_sub!=l],equal_var=False)[1]
        stats[l]['pval']=pvalues
        pvals=np.concatenate((pvals,pvalues))
        pval_startIdx.append(currIdx)
        currIdx+=pvalues.size

    pval_corr=multitest.fdrcorrection(pvals,0.05,'indep')[1]

    for l in np.unique(labels_sub):
        stats[l]['pval_corr']=pval_corr[pval_startIdx[l]:(pval_startIdx[l]+stats[l]['pval'].size)]
#         stats[l].to_csv(os.path.join(nmcodesavepath,str(l)+'_stats05.csv'))
        nmcoSigIdx=np.concatenate((nmcoSigIdx,np.arange(stats[l].shape[0])[np.logical_and(np.abs(stats[l]['avg'])>0.5,np.logical_and(stats[l]['pval_corr']<0.01,np.logical_or(stats[l]['fc']<0.8,stats[l]['fc']>1.2)))]))

    nmcoSigIdx=np.unique(nmcoSigIdx).astype(int)
    print(nmcoSigIdx.size)
    
    df = pd.DataFrame(allstats_nonzero[:,nmcoSigIdx],columns=nmcoNames[nonzeroIdx][nmcoSigIdx])
    corr = df.corr()
    # keep only upper triangle elements (excluding diagonal elements)
    mask_keep = np.triu(np.ones(corr.shape), k=1).astype('bool').reshape(corr.size)
    # melt (unpivot) the dataframe and apply mask
    sr = corr.stack()[mask_keep]
    # filter and get names
    edges = sr[sr > 0.9].reset_index().values[:, :2]
    g = nx.from_edgelist(edges)
    ls_cc = []
    for cc in nx.connected_components(g):
        ls_cc.extend(list(cc))
    for n in nmcoNames[nonzeroIdx][nmcoSigIdx]:
        if n not in ls_cc:
            ls_cc.append(n)

    df_plot=pd.DataFrame(nmcoAvg[:,nonzeroIdx][:,nmcoSigIdx],columns=nmcoNames[nonzeroIdx][nmcoSigIdx])
    df_plot=df_plot[ls_cc]

    fig, ax = plt.subplots(figsize=(100, 10))
    im = ax.imshow(df_plot.to_numpy(),cmap='seismic',vmin=-np.max(np.abs(nmcoAvg[:,nonzeroIdx][:,nmcoSigIdx])),vmax=np.max(np.abs(nmcoAvg[:,nonzeroIdx][:,nmcoSigIdx])))
    ax.set_xticks(np.arange(nmcoSigIdx.size))
    ax.set_xticklabels(ls_cc)
    ax.set_yticks(np.arange(np.unique(labels_sub).size))
    ax.set_yticklabels(np.unique(labels_sub))
    plt.setp(ax.get_xticklabels(), rotation=90, ha="right",rotation_mode="anchor")
    #     fig.set_figheight(35)
    #     fig.set_figwidth(35)
    fig.colorbar(im)
    fig.tight_layout()
    plt.savefig(os.path.join(nmcodesavepath,'all_pval01_fc1.2_avg05_ordered.jpg'))
    plt.close()
    


0
113
1
87
2
88
3
105
4
129
5
97
6
139
7
51


In [65]:
sr

min_calliper         max_calliper                -0.144169
                     smallest_largest_calliper    0.797005
                     min_radius                   0.938358
                     max_radius                  -0.137186
                     mode_radius                  0.919409
                                                    ...   
moments_central-1-1  moments_hu-0                 0.538533
                     moments_hu-1                 0.532140
moments_central-3-1  moments_hu-0                 0.554826
                     moments_hu-1                 0.557562
moments_hu-0         moments_hu-1                 0.989678
Length: 1275, dtype: float64